# Main script to merge PPE, fires, pm25 and wind data

Modules: N/A <br>
Author: Cornelia Ilin <br>
Email: cilin@wisc.edu <br>
Date created: April 27, 2022 <br>

**Citations (data sources)**
    
**Citations (persons)**
1. N/A

**Preferred environment**
1. Code written in Jupyter Notebooks

### Step 1: Import packages

In [ ]:
import pandas as pd
import numpy as np
import os

from pandas.tseries.offsets import DateOffset

import geopandas as gpd
import ast
from shapely.geometry import Point
from geopy.distance import distance
from shapely import wkt
import math
import pyproj
import sklearn.neighbors
dist = sklearn.neighbors.DistanceMetric.get_metric('haversine')

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Step 2: Define working directories

In [ ]:
in_health = 'C:/Users/cilin/Research/CA_hospitals/Input/final_data/health/'
in_fires = 'C:/Users/cilin/Research/CA_hospitals/Input/final_data/fires/'
in_pm25 = 'C:/Users/cilin/Research/CA_hospitals/Input/final_data/pollution/satellite/UW/monthly/'
in_winds = 'C:/Users/cilin/Research/CA_hospitals/Input/final_data/winds/'
out_dir = 'C:/Users/cilin/Research/CA_hospitals/Input/final_data/all_combined/'
#out_dir = 'C:/Users/cilin/Research/CA_hospitals/Figures/'

### Step 3: Define functions

``read data``

In [ ]:
def read_health():
    ''''''
    return pd.read_csv(in_health + 'BPE_final.csv')

In [ ]:
def read_fires():
    ''''''
    return pd.read_csv(in_fires + 'fires_long.csv')

In [ ]:
def read_pm25():
    ''''''
    df = pd.DataFrame()
    # read files
    for file in os.listdir(in_pm25):
        temp_df = pd.read_csv(in_pm25 + file)
        df = pd.concat([df, temp_df], axis=0)
     
    #df = pd.read_csv(in_pm25 + 'pm25_uw_zip_monthly.csv')
    return df

In [ ]:
def read_winds():
    ''''''
    return pd.read_csv(in_winds + 'winds.csv')

``preprocess data``

In [ ]:
def preprocess_health(df, for_df=True):
    '''
    '''
    
    # drop if ZCTA10_centroid is nan()
    df = df[~df['ZCTA10I_centroid'].isna()]
    
    # transform ZCTA10_centroid column to string
    df['ZCTA10I_centroid'] = df['ZCTA10I_centroid'].astype(str)
    
    for col in ['ZCTA10I', 'bthmonthI', 'bthyearI']:
        # transform to string
        df[col] = df[col].astype(str)
        
        # remove .0
        df[col] = df[col].str.split('.').str[0]
        
    # create key with birthyear, birthmonth and birthzip of infant
    df['ZCTA10I_month_year'] = df.ZCTA10I + '_' + df.bthmonthI + '_' + df.bthyearI
    
    # create key with birthyear and birthmonth of infant
    df['bthI_month_year'] = df.bthmonthI+  '_' + df.bthyearI

    if for_df:
        return df
    
    else:
        # keep only cols of interest
        df = df[
            ['ZCTA10I_month_year', 'bthI_month_year', 'ZCTA10I_centroid']
        ]

        # drop duplicates of ZCTA10I_month_year
        df.drop_duplicates(
            subset=['ZCTA10I_month_year'],
            inplace=True
        )

        # transform the centroid column to geometry and then to geopandas
        df['ZCTA10I_centroid']= gpd.GeoSeries.from_wkt(df['ZCTA10I_centroid'])
        gdf = gpd.GeoDataFrame(df, geometry='ZCTA10I_centroid', crs="EPSG:4269")  


        # transform bthI_month_yearto date format
        gdf['bthI_month_year'] =  pd.to_datetime(gdf.bthI_month_year, format="%m_%Y")

        # add 9 months prior birth
        gdf['bthI_month_year_9mbb'] = gdf['bthI_month_year'] - DateOffset(months=9)

        # add 12 months after birth
        gdf['bthI_month_year_12mab'] = gdf['bthI_month_year'] + DateOffset(months=12)

        # transform back to string
        gdf['bthI_month_year'] = gdf['bthI_month_year'].dt.to_period('M').astype(str)
        gdf['bthI_month_year_9mbb'] = gdf['bthI_month_year_9mbb'].dt.to_period('M').astype(str)
        gdf['bthI_month_year_12mab'] = gdf['bthI_month_year_12mab'].dt.to_period('M').astype(str)

        gdf['bthI_month_year'] = gdf['bthI_month_year'].str.split('-').str[1] + "-" + gdf['bthI_month_year'].str.split('-').str[0]
        gdf['bthI_month_year_9mbb'] = gdf['bthI_month_year_9mbb'].str.split('-').str[1] + "-" + gdf['bthI_month_year_9mbb'].str.split('-').str[0]
        gdf['bthI_month_year_12mab'] = gdf['bthI_month_year_12mab'].str.split('-').str[1] + "-" + gdf['bthI_month_year_12mab'].str.split('-').str[0]

        gdf.reset_index(drop=True, inplace=True)


        return gdf

In [ ]:
def preprocess_fires(df):
    '''
    '''
    for col in ['fire_name', 'fire_month', 'fire_year']:
        # transform to string
        df[col] = df[col].astype(str)
        # remove .0
        df[col] = df[col].str.split('.').str[0]
        
    # create key fire_name_year_month
    df['fire_month_year'] = df.fire_name + '_' + df.fire_month + '_' + df.fire_year

    df['month_year'] = df.fire_month + '_' + df.fire_year

    # keep only cols of interest
    df = df[['fire_month_year', 'month_year', 'fire_centroid', 'fire_area_km2',	'fire_duration_days']]

    # transform the centroid column to geometry and then to geopandas
    df['fire_centroid']= gpd.GeoSeries.from_wkt(df['fire_centroid'])
    gdf = gpd.GeoDataFrame(df, geometry='fire_centroid', crs="EPSG:4269")    
    gdf.reset_index(drop=True, inplace=True)
    
    # change format of month_year
    gdf['month_year'] = pd.to_datetime(gdf.month_year, format="%m_%Y")
    gdf['month_year'] = gdf['month_year'].dt.to_period('M').astype(str)
    gdf['month_year'] = gdf['month_year'].str.split('-').str[1] + "-" + gdf['month_year'].str.split('-').str[0]
    
    return gdf

In [ ]:
def preprocess_pm25(df):
    '''
    '''
    # preprocess month to eliminate the 0 for months 1-9
    df['month'] = df.year_month.str[5:7]
    df['month'] = np.where(df.month.str.startswith('0'), df.month.str[1:], df.month)

    # add zcta_month_year column
    df['ZCTA10_month_year'] = df.year_month_zcta.str[8:13] +\
                            '_' + df.month +\
                            '_' + df.year_month_zcta.str[0:4]
    
    # keep only cols of interest
    df = df[['ZCTA10_month_year', 'pm25']]
    
    df.sort_values(
        by=['ZCTA10_month_year'],
        inplace=True
    )
    
    # reset index
    df.reset_index(drop=True, inplace=True)
    
    return df

In [ ]:
def preprocess_winds(df):
    '''
    '''
    for col in ['ZCTA10', 'year_month']:
        df[col] = df[col].astype(str)
    
    # create month and year
    df['year'] = df.year_month.str[0:4]
    df['month'] = df.year_month.str[4:6]
    df['month'] = np.where(df.month.str.startswith('0'), df.month.str[1:], df.month)

    # create zcta_month_year column
    df['ZCTA10_month_year'] = df.ZCTA10 + "_" + df.month + "_" + df.year
    
    
    # keep only cols of interest
    df = df[['ZCTA10_month_year', 'wdir', 'wspd']]
    
    # sort values
    df.sort_values(
        by=['ZCTA10_month_year'],
        inplace=True
    )
    
    # reset index
    df.reset_index(drop=True, inplace=True)     
    
    return df

``impute missing values``

In [ ]:
def impute_missing_PM25(df_h, df_pm):
    ''''''
    # create vars
    df_h['bthmonthI'] = df_h.bthI_month_year.str.split('-').str[0].astype(int)
    df_h['bthyearI'] = df_h.bthI_month_year.str.split('-').str[1].astype(int)
    df_h['ZCTA10I'] = df_h.ZCTA10I_month_year.str.split('_').str[0].astype(int)
    df_h['year_ZCTA10I'] = df_h.ZCTA10I_month_year.str.split('_').str[2] + '_' + df_h.ZCTA10I_month_year.str.split('_').str[0]
    #df_h['ZCTA10I_month'] = df_h.ZCTA10I_month_year.str.split('_').str[0] + '_' + df_h.ZCTA10I_month_year.str.split('_').str[1]
    
    
    
    # merge df_h and df_pm to see which ZCTA10_month_year are missing
    df_hpm = df_h.merge(
        df_pm[['ZCTA10_month_year', 'pm25']],
        left_on='ZCTA10I_month_year',
        right_on='ZCTA10_month_year',
        how='left'
    )
    
    # sort values 
    df_hpm = df_hpm.sort_values(['year_ZCTA10I', 'bthmonthI'])
    df_hpm.reset_index(drop=True, inplace=True)
    
    # take mean of pm25 by year_ZCTA10I and add to df_hpm
    df_hpm_mean = df_hpm.groupby('year_ZCTA10I', as_index=False).pm25.mean()
    df_hpm_mean.rename(columns={'pm25': 'pm25_mean'}, inplace=True)
    
    # take mean of pm25 by bthI_month and add to df_hpm
    df_hpm_mean_z = df_hpm.groupby('ZCTA10I', as_index=False).pm25.mean()
    df_hpm_mean_z.rename(columns={'pm25': 'pm25_mean_z'}, inplace=True)
    
    # merge
    df_hpm = df_hpm.merge(
        df_hpm_mean, 
        on='year_ZCTA10I',
        how='left'
    )

    df_hpm = df_hpm.merge(
        df_hpm_mean_z, 
        on='ZCTA10I',
        how='left'
    )


    # drop if pm_25_mean or pm_25_mean_my is nan()
    df_hpm = df_hpm[~df_hpm.pm25_mean.isna()]
    df_hpm = df_hpm[~df_hpm.pm25_mean_z.isna()]
    df_hpm.reset_index(drop=True, inplace=True)
    
    df_hpm = df_hpm.sort_values(['ZCTA10I', 'bthyearI', 'bthmonthI'])
    df_hpm.reset_index(drop=True, inplace=True)
    
    # if pm25 is missing, substitute with pm25_mean
    #df_hpm['pm25'] = np.where(df_hpm.pm25.isna(), df_hpm.pm25_mean, df_hpm.pm25)
    
    # if pm25 is missing again, subsitute with pm25_mean_my
    #df_hpm['pm25'] = np.where(df_hpm.pm25.isna(), df_hpm.pm25_mean_z, df_hpm.pm25)
    
    # create 9 mmonths before birth pm25 mean value and 12 months after birth pm25 mean value
    def helper(grp):
        ''''''
        pm_9m_lag = pd.concat(
        [
            grp.pm25,
            grp.pm25.shift(1), grp.pm25.shift(2),
            grp.pm25.shift(3), grp.pm25.shift(4),
            grp.pm25.shift(5), grp.pm25.shift(6),
            grp.pm25.shift(7), grp.pm25.shift(8),
            grp.pm25.shift(9)
        ], axis=1).mean(axis=1) # mean by row

        pm_12_fw = pd.concat(
        [
            grp.pm25,
            grp.pm25.shift(-1), grp.pm25.shift(-2),
            grp.pm25.shift(-3), grp.pm25.shift(-4),
            grp.pm25.shift(-5), grp.pm25.shift(-6),
            grp.pm25.shift(-7), grp.pm25.shift(-8),
            grp.pm25.shift(-9), grp.pm25.shift(-10),
            grp.pm25.shift(-11)
        ], axis=1).mean(axis=1) # mean by row

        grp['pm25_9mbb'] = pm_9m_lag   
        grp['pm25_12mab'] = pm_12_fw 
        return grp

    df_hpm = df_hpm.groupby('ZCTA10I', as_index=False).apply(helper)
    
    # add new pm25, pm25_9mbb and pm25_12mab to df_pm
    df_pm = df_pm.drop(columns=['pm25'])
    df_pm = df_pm.merge(
        df_hpm[['ZCTA10_month_year', 'pm25', 'pm25_9mbb', 'pm25_12mab']],
        on='ZCTA10_month_year',
        how='left'
    )
    
    # keep only if 'ZCTA10_month_year' is in df_h
    df_pm = df_pm[df_pm.ZCTA10_month_year.isin(df_h.ZCTA10I_month_year.unique())]
    df_pm.reset_index(drop=True, inplace=True)
    
    return df_pm

In [ ]:
def impute_missing_winds(df_h, df_w):
    ''''''
    # create vars
    df_h['bthmonthI'] = df_h.bthI_month_year.str.split('-').str[0].astype(int)
    df_h['bthyearI'] = df_h.bthI_month_year.str.split('-').str[1].astype(int)
    df_h['ZCTA10I'] = df_h.ZCTA10I_month_year.str.split('_').str[0].astype(int)
    df_h['year_ZCTA10I'] = df_h.ZCTA10I_month_year.str.split('_').str[2] + '_' + df_h.ZCTA10I_month_year.str.split('_').str[0]
    
    
    # merge df_h and df_pm to see which ZCTA10_month_year are missing
    df_hw = df_h.merge(
        df_w[['ZCTA10_month_year', 'wdir', 'wspd']],
        left_on='ZCTA10I_month_year',
        right_on='ZCTA10_month_year',
        how='left'
    )
    
    # sort values 
    df_hw = df_hw.sort_values(['year_ZCTA10I', 'bthmonthI'])
    df_hw.reset_index(drop=True, inplace=True)
    

    # take mean of 'wdir', 'wspd' and add to df_hw
    df_hw_mean = df_hw.groupby('year_ZCTA10I', as_index=False)['wdir', 'wspd'].mean()
    df_hw_mean.rename(columns={'wdir': 'wdir_mean', 'wspd': 'wspd_mean'}, inplace=True)
    
    df_hw = df_hw.merge(
        df_hw_mean, 
        on='year_ZCTA10I',
        how='left'
    )
   

    # drop if wdir_mean or wspd_mean is nan()
    df_hw = df_hw[~df_hw.wdir_mean.isna()]
    df_hw = df_hw[~df_hw.wspd_mean.isna()]
    df_hw.reset_index(drop=True, inplace=True)
    
    df_hw = df_hw.sort_values(['ZCTA10I', 'bthyearI', 'bthmonthI'])
    df_hw.reset_index(drop=True, inplace=True)
    
    # if 'wdir', 'wspd' is missing, substitute with their means
    #df_hw['wdir'] = np.where(df_hw.wdir.isna(), df_hw.wdir_mean, df_hw.wdir)
    #df_hw['wspd'] = np.where(df_hw.wspd.isna(), df_hw.wspd_mean, df_hw.wspd)
    
    # create 9 mmonths before birth and 12 months after birth wdor/wspd values (take mean over the period)
    def helper(grp):
        ''''''
        lag_9m = {}
        fw_12m = {}
        for val in ['wdir', 'wspd']:
            lag_9m[val] = pd.concat(
            [
                grp[val],
                grp[val].shift(1), grp[val].shift(2),
                grp[val].shift(3), grp[val].shift(4),
                grp[val].shift(5), grp[val].shift(6),
                grp[val].shift(7), grp[val].shift(8),
                grp[val].shift(9)
            ], axis=1).mean(axis=1) # mean by row

            fw_12m[val] = pd.concat(
            [
                grp[val],
                grp[val].shift(-1), grp[val].shift(-2),
                grp[val].shift(-3), grp[val].shift(-4),
                grp[val].shift(-5), grp[val].shift(-6),
                grp[val].shift(-7), grp[val].shift(-8),
                grp[val].shift(-9), grp[val].shift(-10),
                grp[val].shift(-11)
            ], axis=1).mean(axis=1) # mean by row
        
        # add columns
        grp['wdir_9mbb'] = lag_9m['wdir']   
        grp['wdir_12mab'] = fw_12m['wdir']
        grp['wspd_9mbb'] = lag_9m['wspd']  
        grp['wspd_12mab'] = fw_12m['wspd'] 
        
        return grp

    df_hw = df_hw.groupby('ZCTA10I', as_index=False).apply(helper)
    
    # add new wspd columns to df_winds
    df_w = df_w.drop(columns=['wspd', 'wdir'])
    df_w = df_w.merge(
        df_hw[['ZCTA10_month_year', 'wdir', 'wdir_9mbb', 'wdir_12mab', 'wspd', 'wspd_9mbb', 'wspd_12mab']],
        on='ZCTA10_month_year',
        how='left'
    )
    
    # keep only if 'ZCTA10_month_year' is in df_h
    df_w = df_w[df_w.ZCTA10_month_year.isin(df_h.ZCTA10I_month_year.unique())]
    df_w.reset_index(drop=True, inplace=True)
    
    return df_w

``distance``

In [ ]:
def compute_distance_bearing(gdf_h, gdf_f, time_period):
    ''' Merge fire and health data and compute distance between fires centroid and birth zip centroid
    '''
    gdf = gdf_h.groupby(
        ['ZCTA10I_month_year'],
        as_index=False
    ).apply(helper, gdf_f, time_period)

    # reset index
    gdf.reset_index(drop=True, inplace=True)
    
    return gdf

In [ ]:
def helper(grp, gdf_f, time_period):
    '''
    '''
    # find month year in health data (grp)
    month_year = grp.bthI_month_year.unique()[0]
    month_year_9mbb = grp.bthI_month_year_9mbb.unique()[0]
    month_year_12mab = grp.bthI_month_year_12mab.unique()[0]
    
    if time_period=='9mbb':
        month_year_range = pd.date_range(
            month_year_9mbb, month_year, freq='MS'
        ).strftime("%m-%Y").tolist()
        
    if time_period=='12mab':
        month_year_range = pd.date_range(
            month_year, month_year_12mab, freq='MS'
        ).strftime("%m-%Y").tolist()
        

    # grab only fires in this month and year
    if time_period=='current':
        temp_fires = gdf_f[gdf_f.month_year.eq(month_year)]
    else:
        temp_fires = gdf_f[gdf_f.month_year.isin(month_year_range)]
    temp_fires.reset_index(drop=True, inplace=True)
    #print(temp_fires.shape)
 
    if temp_fires.shape[0]:
        # add variable to indicate there were fires in this month and year
        grp['fires_in_bthI_month_year'] = 'yes'
        # replicate grp to match size of fires
        grp = pd.concat([grp]*temp_fires.shape[0], ignore_index=True)
        # match grp and fires
        grp = pd.concat([grp, temp_fires], axis=1)

        # compute distance and bearing angle between each fire centroid and birth zip code centroid #
        #############################################################################################
        # bearing angle intuition: https://www.mathsteacher.com.au/year7/ch08_angles/07_bear/bearing.htm
        # beearing angle online tool: https://www.igismap.com/map-tool/bearing-angle
        # bearing angle online tool: https://www.movable-type.co.uk/scripts/latlong.html
        geodesic = pyproj.Geod(ellps='WGS84')
        fwd_azimuth, back_azimuth, distance = geodesic.inv(
            grp.fire_centroid.x, grp.fire_centroid.y, # this is for fire centroid (lon, lat); # notice that this lines defines the fire centroid as the starting point
            grp.ZCTA10I_centroid.x, grp.ZCTA10I_centroid.y # this is for zip centroid (lon, lat)
        )
        
        # define bearing_angle as the fwd_azimuth
        grp['bearing_angle'] = fwd_azimuth
        
        # Note that the bearing angle is btw -180° and + 180°, we want to transoform it to a compas bearing from 0 to 360
        # See also: https://towardsdatascience.com/calculating-the-bearing-between-two-geospatial-coordinates-66203f57e4b4
        grp['bearing_angle'] = (grp.bearing_angle + 360) % 360 # degrees from the north;
        
        # distance is originally reported in meters; divide by 1000 to transform in km
        grp['fire_ZCTA10I_dist'] = distance/1000  
        
    else:
        # add variable to indicate there were no fires in this month and year
        grp['fires_in_bthI_month_year'] = 'no'
    
    return grp

``merge``

In [ ]:
def merge_dfs(gdf_hf, df_p, df_w):
    ''''''
    
    # merge health and pm25
    gdf = gdf_hf.merge(
        df_p,
        left_on='ZCTA10I_month_year',
        right_on='ZCTA10_month_year',
        how='left'
    )
    
    # drop to avoid duplicates with df_w
    gdf.drop(columns=['ZCTA10_month_year'], inplace=True)
    
    # merge health + pm25 and winds
    gdf = gdf.merge(
        df_w, 
        left_on='ZCTA10I_month_year',
        right_on='ZCTA10_month_year',
        how='left'
    )
    
    
    return gdf

``add instrument``

In [ ]:
def add_instrument(gdf, time_period):
    ''''''
    ## for wind direction ##
    ########################
    if time_period=='current':
        windcol = 'wdir'
    if time_period=='9mbb':
        windcol = 'wdir_9mbb'
    if time_period=='12mab':
        windcol = 'wdir_12mab'


    # add compas bins for wind direction
    gdf['binned_wdir']=pd.cut(
        x=gdf[windcol],
        bins=np.arange(0,390,30) # 12 bins in total
    )

    # add dummies for binned wind direction
    gdf = pd.concat(
        [gdf, pd.get_dummies(gdf.binned_wdir, prefix='bwdir')],
        axis=1
    )


    ## for bearing angles ##
    ########################
    # add compas bins for bearing angle between fires and birth zip code centroid;
    gdf['binned_bearing']=pd.cut(
        x=gdf.bearing_angle,
        bins=np.arange(0,390,30) # 12 bins in total
    )

    # add dummies for binned compass bins
    gdf = pd.concat(
        [gdf, pd.get_dummies(gdf.binned_bearing, prefix='bfire')],
        axis=1
    )
    
    
    ## create wildfire instrument ##
    ################################
    # define bins in gdf
    bins12 = [
        '_(0, 30]', '_(30, 60]', '_(60, 90]',
        '_(90, 120]', '_(120, 150]', '_(150, 180]',
        '_(180, 210]', '_(210, 240]', '_(240, 270]',
        '_(270, 300]', '_(300, 330]', '_(330, 360]'
    ]

    bins9 = [
        '_(0, 45]', '_(45, 90]', '_(90, 135]',
        '_(135, 180]', '_(180, 225]', '_(225, 270]',
        '_(270, 315]', '_(315, 360]',
    ]


    # compute weighed wildfire exposure for each bin
    for val in bins12:
        # wind direction matches bearing is 1
        gdf['wfe'+val] = gdf['bwdir'+val].astype(int) * gdf['bfire'+val].astype(int) 

        # account for wind speed and distance between fire and zip at birth
        #gdf_hfpw['wfe'+val] = gdf_hfpw['wfe'+val] * gdf_hfpw.wspd/gdf_hfpw.fire_ZCTA10I_dist
        gdf['wfe'+val] = gdf['wfe'+val]/gdf.fire_ZCTA10I_dist

    # add instrument (sum over wildfire exposure (wfe) columns)
    cols = [col for col in gdf.columns if col.startswith('wfe')]
    gdf['wfe'] = gdf[cols].sum(axis=1)

    # take average across all fires for a given ZCTA10I_month_year
    gdf = gdf.groupby(['ZCTA10I_month_year'], as_index=False).wfe.sum()
    
    # rename wfe column
    if time_period in ['9mbb', '12mab']:
        gdf.rename(columns={'wfe':'wfe'+'_'+time_period}, inplace=True)

    gdf.reset_index(
        drop=True, 
        inplace=True
    )
    'ZCTA10I_centroid', 'ZCTA10I'
    # keep only cols of interest
    keep_cols = [val for val in gdf.columns if val.startswith(('ZCTA10I','wfe'))]
    gdf = gdf[keep_cols]
    
    return gdf

---
### Step 4: Read and preprocess data

In [ ]:
df_health = preprocess_health(read_health())

In [ ]:
gdf_health= preprocess_health(df_health, for_df=False)
gdf_health.head(2)

In [ ]:
gdf_fires = preprocess_fires(read_fires())
gdf_fires.head(2)

In [ ]:
df_pm25 = preprocess_pm25(read_pm25())
df_pm25.head(2)

In [ ]:
df_winds = preprocess_winds(read_winds())
df_winds.head(2)

---
### Step 5: Impute missing values

<span style="color:chocolate">for PM25, and add averaged for 9 months before and 12 months after birth</span>

In [ ]:
df_pm25 = impute_missing_PM25(gdf_health, df_pm25)
df_pm25.head(2)

<span style="color:chocolate">for winds, and add averaged for 9 months before and 12 months after birth</span>

In [ ]:
df_winds = impute_missing_winds(gdf_health, df_winds)
df_winds.head(2)

---
### Step 6: Compute distance between fires and health zip code centroid

<span style="color:chocolate">for current month of birth, and for 9 months before and 12 months after birth</span>

In [ ]:
# for month and year of birth
%time gdf_hf = compute_distance_bearing(gdf_health, gdf_fires, 'current')
gdf_hf.head(2)

In [ ]:
# for 9 months before birth
%time gdf_hf_9mbb = compute_distance_bearing(gdf_health, gdf_fires, '9mbb')
gdf_hf_9mbb.head(2)

In [ ]:
# for 12 months after birth
%time gdf_hf_12mab = compute_distance_bearing(gdf_health, gdf_fires, '12mab')
gdf_hf_12mab.head(2)

IMPORTANT: Note that some month_year (current, 9 months before or 12 months after birth) did not have a fire. Check fires_in_bthI_month_year

---
### Step 7: Merge health, pm25, winds, and fires

<span style="color:chocolate">for current month of birth, and for 9 months before and 12 months after birth</span>

In [ ]:
gdf_hfpw = merge_dfs(gdf_hf, df_pm25, df_winds)
gdf_hfpw_9mbb = merge_dfs(gdf_hf_9mbb, df_pm25, df_winds)
gdf_hfpw_12mab = merge_dfs(gdf_hf_12mab, df_pm25, df_winds)

---
### Step 8: Add wildfire instrument

<span style="color:chocolate">for current month of birth, and for 9 months before and 12 months after birth</span>

In [ ]:
gdf_hfpwi = add_instrument(gdf_hfpw, time_period='current')
gdf_hfpwi_9mbb = add_instrument(gdf_hfpw_9mbb, time_period='9mbb')
gdf_hfpwi_12mab = add_instrument(gdf_hfpw_12mab, time_period='12mab')

# merge all 3
gdf_hfpwi = gdf_hfpwi.merge(
    gdf_hfpwi_9mbb,
    on='ZCTA10I_month_year',
    how='left'
)


gdf_hfpwi = gdf_hfpwi.merge(
    gdf_hfpwi_12mab,
    on='ZCTA10I_month_year',
    how='left'
)

gdf_hfpwi.head(2)

---
### Step 9: Add to health data

the data is at the patient level

In [ ]:
### Step 9: Merge with original health data
# merge to original health data, df_health
gdf_hfpwi = df_health.merge(
    gdf_hfpwi,
    on=['ZCTA10I_month_year'],
    how='left'
)

# add pm25 again
gdf_hfpwi = gdf_hfpwi.merge(
        df_pm25,
        left_on='ZCTA10I_month_year',
        right_on='ZCTA10_month_year',
        how='left'
    )

# rename pm25 and wfe
gdf_hfpwi.rename(
    columns={'pm25':'pm25I',
             'pm25_9mbb': 'pm25I_9mbb',
             'pm25_12mab': 'pm25I_12mab',
             'wfe':'wfeI',
             'wfe_9mbb':'wfeI_9mbb',
             'wfe_12mab':'wfeI_12mab'
            },
    inplace=True
)

gdf_hfpwi.head(2)

#### Step 9: Export to .csv

In [ ]:
gdf_hfpwi.to_csv(out_dir + 'analysis_data_birth_pdd_edd.csv')

In [ ]:
%matplotlib inline
from icd9cms.icd9 import search
temp = gdf_hfpwi.groupby('diagI00', as_index=False).data_source.count()
temp.sort_values('data_source', ascending=False, inplace=True)
temp.reset_index(drop=True, inplace=True)
temp['diagI00_3d'] = temp.diagI00.str[:3]
temp

In [ ]:
## find names of diagnosis in vocab ##
######################################
diag_vocab = temp[temp.diagI00_3d.str.startswith('9')].diagI00_3d.unique()
diag_3d_dict = {}

for val in diag_vocab:
    try:
        code = str(search(val)).split(':')[:2] # search() function is from icd9cms.icd9
        diag_3d_dict[code[0]] = code[1]
    except:
        # if diag code is not in icd9cms.icd9, continue
        continue
diag_3d_dict

In [ ]:
temp[temp.diagI00_3d.str.startswith('54')]

In [ ]:
temp[temp.diagI00_3d.str.startswith('9')]